In [1]:
import pandas as pd
import requests
from datetime import datetime
import tabula

# Importing public resources files

In [2]:
df_public_resources = pd.read_csv("public_resources_list.csv")

In [33]:
links_pax_entries_stations = df_public_resources.query(
    " file_type == 'csv' & resource_name.str.contains('Entrada de Passageiros por Estação - Média Dias Úteis') "
    )
links_pax_entries_stations

,resource_name,resource_link,file_type,mod_date
3,Entrada de Passageiros por Estação - Média Dia...,https://transparencia.metrosp.com.br/sites/def...,csv,2023-05-09
10,Entrada de Passageiros por Estação - Média Dia...,https://transparencia.metrosp.com.br/sites/def...,csv,2023-05-09
17,Entrada de Passageiros por Estação - Média Dia...,https://transparencia.metrosp.com.br/sites/def...,csv,2023-05-09


In [10]:
# Mapping the months to its numeric equivalent
month_map = {
    'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4,
    'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8,
    'set': 9, 'out': 10, 'nov': 11, 'dez': 12
}

# Complete PES wrangling for L01 - Blue

In [27]:
def l01_pes(url, year):
    # Read the CSV file
    df_raw = pd.read_csv(
        url, 
        encoding="latin-1", 
        sep=";",
        skiprows=5,
        skipfooter=166,
        usecols=range(0, 13),
        engine='python',
        )
    # Reshape the DataFrame using melt    
    df_raw = df_raw.melt(id_vars=["Estação"], var_name="month", value_name="dpea")
    # Remove "*" from the month column
    df_raw["month"] = df_raw["month"].str.replace(r"\*", "", regex=True)
    # Convert month names to datetime objects
    df_raw["month"] = df_raw["month"].apply(lambda x: pd.to_datetime(year + str(month_map[x.lower()]), format="%Y%m"))
    # Rename the "Estação" column to "station"
    df_raw.rename(columns={"Estação": "station"}, inplace=True)
    # Add a "line" column with value 1
    df_raw["line"] = 1
    # Reorder the columns
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])
    
    return df_raw

In [28]:
results_l01_pes = []
for index, row in links_pax_entries_stations.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l01_pes(resource_link, year)
    results_l01_pes.append(df_processed)
    

In [42]:
l01_pes_complete = pd.concat(results_l01_pes, ignore_index=True).sort_values("month")
l01_pes_complete

,month,line,station,dpea
298,2021-01-01,1,Tucuruvi,36.0
297,2021-01-01,1,Parada Inglesa,7.0
296,2021-01-01,1,Jardim São Paulo-Ayrton Senna,6.0
295,2021-01-01,1,Santana,33.0
294,2021-01-01,1,Carandiru,6.0
...,...,...,...,...
256,2023-12-01,1,Saúde,NaN
255,2023-12-01,1,São Judas,NaN
254,2023-12-01,1,Conceição,NaN
274,2023-12-01,1,Parada Inglesa,NaN


In [43]:
l01_pes_complete.to_csv("final_datasets/l01_pes_complete.csv", index=False)

# Complete PES wrangling for L02 - Green

In [46]:
def l02_pes(url, year):
    df_raw = pd.read_csv(
        url, 
        encoding="latin-1", 
        sep=";",
        skiprows=35,
        skipfooter=145,
        usecols=range(0, 13),
        engine='python',
        )
    df_raw = df_raw.melt(id_vars=["Estação"], var_name="month", value_name="dpea")
    df_raw["month"] = df_raw["month"].str.replace(r"\*", "", regex=True)
    df_raw["month"] = df_raw["month"].apply(lambda x: pd.to_datetime(year + str(month_map[x.lower()]), format="%Y%m"))
    df_raw.rename(columns={"Estação": "station"}, inplace=True)
    # Remove ¹ and ² characters
    df_raw['station'] = df_raw['station'].str.replace(r'[¹²]', '', regex=True)
    df_raw["line"] = 2
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])
    
    return df_raw

In [47]:
results_l02_pes = []
for index, row in links_pax_entries_stations.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l02_pes(resource_link, year)
    results_l02_pes.append(df_processed)

In [48]:
l02_pes_complete = pd.concat(results_l02_pes, ignore_index=True).sort_values("month")
l02_pes_complete

,month,line,station,dpea
181,2021-01-01,2,Vila Madalena,10.0
180,2021-01-01,2,Santuário N.S. de Fátima-Sumaré,5.0
179,2021-01-01,2,Clínicas,13.0
178,2021-01-01,2,Consolação,49.0
177,2021-01-01,2,Trianon-Masp,21.0
...,...,...,...,...
162,2023-12-01,2,Brigadeiro,NaN
163,2023-12-01,2,Trianon-Masp,NaN
164,2023-12-01,2,Consolação,NaN
166,2023-12-01,2,Santuário N.S. de Fátima-Sumaré,NaN


In [49]:
l02_pes_complete.to_csv("final_datasets/l02_pes_complete.csv", index=False)

# Complete PES wrangling for L03 - red

In [51]:
def l03_pes(url, year):
    df_raw = pd.read_csv(
        url, 
        encoding="latin-1", 
        sep=";",
        skiprows=56,
        skipfooter=120,
        usecols=range(0, 13),
        engine='python',
        )
    df_raw = df_raw.melt(id_vars=["Estação"], var_name="month", value_name="dpea")
    df_raw["month"] = df_raw["month"].str.replace(r"\*", "", regex=True)
    df_raw["month"] = df_raw["month"].apply(lambda x: pd.to_datetime(year + str(month_map[x.lower()]), format="%Y%m"))
    df_raw.rename(columns={"Estação": "station"}, inplace=True)
    # Remove ¹ and ² characters
    df_raw['station'] = df_raw['station'].str.replace(r'[¹²]', '', regex=True)
    df_raw["line"] = 3
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])
    
    return df_raw

In [52]:
results_l03_pes = []
for index, row in links_pax_entries_stations.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l03_pes(resource_link, year)
    results_l03_pes.append(df_processed)

In [56]:
l03_pes_complete = pd.concat(results_l03_pes, ignore_index=True).sort_values("month")
l03_pes_complete

,month,line,station,dpea
233,2021-01-01,3,Palmeiras-Barra Funda,95.0
232,2021-01-01,3,Marechal Deodoro,18.0
231,2021-01-01,3,Santa Cecília,16.0
230,2021-01-01,3,República,72.0
229,2021-01-01,3,Anhangabaú,32.0
...,...,...,...,...
201,2023-12-01,3,Guilhermina-Esperança,NaN
200,2023-12-01,3,Patriarca,NaN
199,2023-12-01,3,Artur Alvim,NaN
205,2023-12-01,3,Tatuapé,NaN


In [57]:
l03_pes_complete.to_csv("final_datasets/l03_pes_complete.csv", index=False)

# Complete PES wrangling for L15 - silver

In [59]:
def l15_pes(url, year):
    df_raw = pd.read_csv(
        url, 
        encoding="latin-1", 
        sep=";",
        skiprows=80,
        skipfooter=103,
        usecols=range(0, 13),
        engine='python',
        )
    df_raw = df_raw.melt(id_vars=["Estação"], var_name="month", value_name="dpea")
    df_raw["month"] = df_raw["month"].str.replace(r"\*", "", regex=True)
    df_raw["month"] = df_raw["month"].apply(lambda x: pd.to_datetime(year + str(month_map[x.lower()]), format="%Y%m"))
    df_raw.rename(columns={"Estação": "station"}, inplace=True)
    # Remove ¹ and ² characters
    df_raw['station'] = df_raw['station'].str.replace(r'[¹²]', '', regex=True)
    df_raw["line"] = 15
    df_raw = df_raw.reindex(columns=["month", "line", "station", "dpea"])
    
    return df_raw

In [60]:
results_l15_pes = []
for index, row in links_pax_entries_stations.iterrows():
    resource_link = row["resource_link"]
    year = row["resource_name"].split(" - ")[-1]
    df_processed = l15_pes(resource_link, year)
    results_l15_pes.append(df_processed)

In [63]:
l15_pes_complete = pd.concat(results_l15_pes, ignore_index=True).sort_values("month")
l15_pes_complete

,month,line,station,dpea
142,2021-01-01,15,Jardim Colonial,0.0
141,2021-01-01,15,São Mateus,11.0
140,2021-01-01,15,Fazenda da Juta,2.0
139,2021-01-01,15,Sapopemba,5.0
138,2021-01-01,15,Jardim Planalto,2.0
...,...,...,...,...
126,2023-12-01,15,Vila União,NaN
127,2023-12-01,15,Jardim Planalto,NaN
128,2023-12-01,15,Sapopemba,NaN
130,2023-12-01,15,São Mateus,NaN


In [64]:
l15_pes_complete.to_csv("final_datasets/l15_pes_complete.csv", index=False)